# Sensor Synchronization

In [1]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit as bp
import empkins_io as eio
import empkins_micro as emic
import empkins_macro as emac

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

[(0.0, 0.2196078431372549, 0.396078431372549),
 (0.596078431372549, 0.6431372549019608, 0.6823529411764706),
 (0.788235294117647, 0.5764705882352941, 0.07450980392156863),
 (0.0, 0.6941176470588235, 0.9215686274509803),
 (0.0, 0.6078431372549019, 0.4666666666666667),
 (0.5529411764705883, 0.0784313725490196, 0.1607843137254902)]

In [3]:
subject_id = "VP_02"
condition = "control"
study_part = "pre"

study_part_dict = {
    "pre": 0,
    "mist": 1,
    "post": 2
}

outlier_correction = ["statistical_rr", "statistical_rr_diff", "artifact", "physiological"]

In [4]:
base_path = Path("../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject/{}/{}".format(subject_id, condition))

ecg_path = data_path.joinpath("ecg/raw")
radar_path = data_path.joinpath("radar/raw")
video_path = data_path.joinpath("video")
timelog_path = data_path.joinpath("time_log/processed")

In [5]:
ecg_files = list(sorted(ecg_path.glob("*.bin")))
radar_files = list(sorted(radar_path.glob("*.mat")))
video_files = list(sorted(video_path.glob("*.mp4")))

ecg_file = ecg_files[study_part_dict[study_part]]
radar_file = radar_files[study_part_dict[study_part]]
video_file = video_files[study_part_dict[study_part]]

timelog_file = timelog_path.joinpath("time_log_{}.csv".format(subject_id))
print("{}\t {}\t {}".format(ecg_file.name, radar_file.name, video_file.name))

NilsPodX-E18A_20210610_114109.bin	 2021-06-10_11-41-30_VP_02_KB_1__rawdata.mat	 VP_02_MIST_H.264.mp4


## Load Time Log

In [6]:
time_intervals = bp.io.load_time_log(timelog_file, continuous_time=False)
time_intervals = time_intervals.filter(like=study_part)
time_intervals

phase         pre           pre_baseline          
time        start       end        start       end
subject                                           
VP_02    11:40:27  11:59:46     11:49:09  11:51:10

## Load NilsPod and Radar Data

In [7]:
ecg, fs_ecg = bp.io.nilspod.load_dataset_nilspod(ecg_file)
# flip signal because it's upside down
# ecg["ecg"]= -1 * ecg["ecg"]

In [8]:
radar, fs_radar = eio.io.a04.load_data(radar_file, datastreams=["hr"])
radar = radar["hr"]

### Align both Signals

In [9]:
radar, ecg = eio.sensors.utils.align_to_reference_data(radar, ecg)

### Process ECG Signal

In [10]:
ep = bp.signals.ecg.EcgProcessor(data=ecg, sampling_rate=fs_ecg, time_intervals=time_intervals)
ep.ecg_process(outlier_correction=outlier_correction)

ecg_dict = ep.rpeaks

  0%|          | 0/2 [00:00<?, ?it/s]

### Process Radar Signal

In [11]:
rp = eio.sensors.a04.A04Processor(radar, time_intervals=time_intervals)
rp.process(outlier_correction=outlier_correction)

radar_dict = rp.rpeaks

  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
key = "pre_baseline"

hr_ecg = ecg_dict[key]
hr_radar = radar_dict[key]

### ECG Overview Plot

In [22]:
fig, axs = bp.signals.ecg.plotting.ecg_plot(ep, key=key, figsize=(10,5))
fig.savefig("img_fau_colors.png", transparent=True, dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FacultyColors(fau='#002F6C', tech='#779FB5', phil='#FFB81C', med='#00A3E0', nat='#43B02A', wiso='#C8102E')


## Sync ECG and Radar

### Resample Signals to 1Hz

In [ ]:
hr_ecg_res = bp.utils.data_processing.resample_sec(hr_ecg["Heart_Rate"])
hr_radar_res = bp.utils.data_processing.resample_sec(hr_radar["Heart_Rate"])

index_intersect = hr_ecg_res.index.intersection(hr_radar_res.index)

hr_ecg_res = hr_ecg_res.loc[index_intersect]
hr_radar_res = hr_radar_res.loc[index_intersect]

### Align Signals

In [ ]:
hr_ecg_res, hr_radar_shift, shift_idx = eio.signal_alignment.signal_align(hr_ecg_res, hr_radar_res)

print("Shift: {:.3f} seconds".format(shift_idx))

### Before Shift

In [ ]:
fig, ax = plt.subplots()
eio.sensors.a04.plotting.hr_plot_ecg_radar(hr_ecg=hr_ecg_res, hr_radar=hr_radar_res, ax=ax)

fig.tight_layout()

#### Sensor Data Agreement

In [ ]:
eio.sensor_validation.plotting.plot_sensor_agreement(
    reference=hr_ecg_res["Heart_Rate"], 
    target=hr_radar_res["Heart_Rate"], 
    reference_name="$HR_{ECG}$ [bpm]", 
    target_name="$HR_{Radar}$ [bpm]"
)

In [ ]:
metric_dict = eio.sensor_validation.metrics.agreement_metrics(hr_ecg_res["Heart_Rate"], hr_radar_res["Heart_Rate"])
eio.sensor_validation.metrics.display_metrics(metric_dict)

### After Shift

In [ ]:
fig, ax = plt.subplots()
eio.sensors.a04.plotting.hr_plot_ecg_radar(hr_ecg=hr_ecg_res, hr_radar=hr_radar_shift, plot_outlier=False, plot_radar_quality=False, ax=ax)

fig.tight_layout()

#### Sensor Data Agreement

In [ ]:
eio.sensor_validation.plotting.plot_sensor_agreement(
    reference=hr_ecg_res["Heart_Rate"], 
    target=hr_radar_shift["Heart_Rate"], 
    reference_name="$HR_{ECG}$ [bpm]", 
    target_name="$HR_{Radar}$ [bpm]"
)

In [ ]:
metric_dict = eio.sensor_validation.metrics.agreement_metrics(hr_ecg_res["Heart_Rate"], hr_radar_shift["Heart_Rate"])
eio.sensor_validation.metrics.display_metrics(metric_dict)